<a href="https://colab.research.google.com/github/Jegovila/cursoVR/blob/main/Practica9%3A%20Descriptores/python/Pr%C3%A1ctica9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica9%3A%20Descriptores/python/I1.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica9%3A%20Descriptores/python/I2.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica9%3A%20Descriptores/python/I3.jpg"

In [2]:
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# Parte 1

In [ ]:
I1 = cv2.imread("I1.jpg")
I2 = cv2.imread("I2.jpg")

I1 = cv2.resize(I1, (0,0), fx=0.2, fy=0.2)
I2 = cv2.resize(I2, (0,0), fx=0.2, fy=0.2)

fig, prev = plt.subplots(1, 2, figsize=(10,7))
fig.tight_layout()
plt.clf()

plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(I1, cv2.COLOR_BGR2RGB))
plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(I2, cv2.COLOR_BGR2RGB))

Elegir el algoritmo descriptor

In [ ]:
descriptor = cv2.SIFT_create()
#descriptor = cv2.xfeatures2d.SURF_create()
#descriptor = cv2.ORB_create()
#descriptor = cv2.AKAZE_create()

Extraer puntos de interés y calcular sus vectores descriptores

In [ ]:
kpts1, desc1 = descriptor.detectAndCompute(I1, None)
kpts2, desc2 = descriptor.detectAndCompute(I2, None)

print("Keypoints en imagen 1: ", len(kpts1))
print(desc1.shape[0], " descriptores en imagen 1, de tamaño: ", desc1.shape[1])

print("Keypoints en imagen 2: ", len(kpts2))
print(desc2.shape[0], " descriptores en imagen 2, de tamaño: ", desc2.shape[1])

Hacer matches

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
matches = bf.match(desc1, desc2)

Ordenarlos por distancia y mostrar con drawMatches los mejores

ej. matches[:10].

In [ ]:
matches = sorted(matches, key=lambda x: x.distance)

In [ ]:
corr = 0
print("Hay ",len(matches), "matches")
print("El match ", corr, " está conformado por los keyponts -> kpts1: ", matches[corr].queryIdx, " y ", matches[corr].trainIdx )
print("El keypoint ", matches[corr].queryIdx, " de la imagen 1 con coordenadas Kpts1: ", kpts1[matches[corr].queryIdx].pt)
print("corresponde al keypoint ", matches[corr].trainIdx, " de la imagen 2 con coordenadas Kpts2: ", kpts2[matches[corr].trainIdx].pt)

Desplegar resultados

In [ ]:
I3 = cv2.drawMatches(I1, kpts1, I2, kpts2, matches[:20], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2_imshow(I3)

# Parte 2: Imagen panorámica

In [3]:
from imutils import paths
import numpy as np
import matplotlib.pyplot as plt

## Función para mostrar las imágenes

In [4]:
def plot_images(*imgs, figsize=(10,5), hide_ticks=False):
    '''Display one or multiple images.'''
    f = plt.figure(figsize=figsize)
    width = np.ceil(np.sqrt(len(imgs))).astype('int')
    height = np.ceil(len(imgs) / width).astype('int')
    for i, img in enumerate(imgs, 1):
        ax = f.add_subplot(height, width, i)
        if hide_ticks:
            ax.axis('off')
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

## Inicialización

In [5]:
imagePaths = sorted(list(paths.list_images("./")))
images = []
images_gray = []
kpts = []
desc = []

akaze = cv2.AKAZE_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

In [ ]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)

    scale_percent = 20  # Reducir a este porcentaje
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    image = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

    images.append(image)

    keypoints, features = akaze.detectAndCompute(image, None)
    kpts.append(keypoints)
    desc.append(features)
plot_images(images[0], images[1])

In [ ]:
matches = bf.match(desc[0], desc[1])
matches = sorted(matches, key=lambda x: x.distance)

keypoints_drawn_left = cv2.drawKeypoints(images[0], kpts[0], None, color=(0, 0, 255))
keypoints_drawn_right = cv2.drawKeypoints(images[1], kpts[1], None, color=(0, 0, 255))

plot_images(images[0], keypoints_drawn_left, images[1], keypoints_drawn_right)

In [ ]:
matches_drawn = cv2.drawMatches(images[0], kpts[0], images[1], kpts[1], matches[:100], None, matchColor=(0,0,255), flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
plot_images(matches_drawn)

In [9]:
left_pts = []
right_pts = []
for m in matches[:100]:
    l = kpts[0][m.queryIdx].pt
    r = kpts[1][m.trainIdx].pt
    left_pts.append(l)
    right_pts.append(r)

In [10]:
M, _ = cv2.findHomography(np.float32(right_pts), np.float32(left_pts))
dim_x = images[0].shape[1] + images[1].shape[1]
dim_y = max(images[0].shape[0]+100, images[1].shape[0]+100)
dim = (dim_x, dim_y)

In [ ]:
warped = cv2.warpPerspective(images[1], M, dim)
plot_images(warped)
comb = warped.copy()
# combinar las dos imagenes
comb[0:images[0].shape[0],0:images[0].shape[1]] = images[0]
# crop
r_crop = 1300
comb = comb[:, :r_crop]
plot_images(comb)